## Particle within radius or overlap calculations

In [1]:
#user-defined
import ipynb.fs.full.helper as hp

#dataframes
import pandas as pd
import h5py

#speedup
import multiprocessing as mp

#suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.TimeSeries = pd.Series 

#math
import numpy as np
import math as m
from scipy.spatial.distance import cdist


#plots
import pylab as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar
import seaborn as sns

#misc
import time
np.set_printoptions(suppress=True)

In [2]:
# set_number = 5
dir_save = '../../data/Sprints/AN/'
dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/LowRes/'
# dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/Run01/'
df1 = pd.read_hdf(dir+'Run01_low.h5')
df2 = pd.read_hdf(dir+'Run02_low.h5')
df3 = pd.read_hdf(dir+'Run03_low.h5')
df4 = pd.read_hdf(dir+'Run04_low.h5')
df5 = pd.read_hdf(dir+'Run05_low.h5')
# df2 = pd.read_hdf(dir+'Run02/Run02Okay.h5')

In [3]:
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)
# df = pd.read_hdf(dir+'Run05/Run05_InterpolatedX.hdf')
# df.drop(index=df1.index[:6000], axis=0, inplace=True)
df.reset_index(inplace=True, drop=True) 

### Calculate Expected encounters

In [4]:
sync_time = df.master_time-df.master_time[0]
dt=df.master_time[1]-df.master_time[0]
df.insert(1,'sync_time',sync_time)

# storing the odor location in a numpy array to avoid reading pandas
source = np.array([[0,0]])
odor_position = np.array([[df.xsrc[i],df.ysrc[i]] for i in range (len(df.xsrc))]) 

distance_from_source = np.array([cdist(odor_position,source)]).flatten()

### Is Encountered odor particle within the Calculated particle radius?

The loop below computes locations of the eastwest and northsouth particles and compares with a radius (distance) to interpret if the encountered particle were close to calculated particle. The execution for the loop is 23.56 seconds for 59770 rows of data

For stationery wind sensor: The loop below takes in ith eastwest and subtracts the ith U and V elements to get the i-1 state, as the eastwest is summation of all the wind positions. 

[3 + 2 + 1] = [6]

hence ith _state = 6

_state = _state-3

_state = _state-2




In [5]:
def wrapped_angular_diff(a, b):
    return np.arctan2(np.sin(a-b), np.cos(a-b))

def relative_motion_calculation(streakline, pos,i):
    if(pos == 0):
        x = streakline[pos:(pos+2),0]
        y = streakline[pos:(pos+2),1]
    else:
        x = streakline[(pos-1):(pos+2),0]
        y = streakline[(pos-1):(pos+2),1]

    slope_streakline,intercept_streakline=np.polyfit(x,y,1)
    slope_sensor = np.arctan2(df.gps_linear_y[i],df.gps_linear_x[i])
    
    relative_motion.append(wrapped_angular_diff(slope_streakline,slope_sensor))

In [6]:
def compute_distance(streakline, odor_position,i):
    source = np.array([[0,0]])
    streakline = np.concatenate((streakline,source))
    distance = cdist(odor_position,streakline).flatten()   # cdist compares distance for all the points in both arrays
    nearest_from_streakline.append(np.min(distance))
    pos = np.where(distance == np.amin(distance))
    relative_motion_calculation(streakline, pos[0][0],i)
    
    if (len(pos[0]) > 1):
        length=(np.sqrt(np.sum(np.diff(np.array(streakline[int(pos[0][0]):]), axis=0)**2, axis=1)))
    else:
        length=(np.sqrt(np.sum(np.diff(np.array(streakline[int(pos[0]):]), axis=0)**2, axis=1)))
        
    distance_along_streakline.append(np.sum(length))

In [7]:
def find_streakline():
    eastwest = [np.sum(df.U[j:])*dt for j in range(0,len(df))]
    northsouth = [np.sum(df.V[j:])*dt for j in range(0,len(df))]
    return eastwest,northsouth

In [8]:
eastwest,northsouth =find_streakline()

In [13]:
len(eastwest)-30000

71708

In [21]:
nearest_from_streakline = []
distance_along_streakline = []
relative_motion=[]

for i in range(len(eastwest)-1, 50000, -1):
    odor_pos = [odor_position[i]] 
    eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    compute_distance(wind_pos,odor_pos,i)

In [22]:
for i in range(len(eastwest)-1, 6000, -1):
    odor_pos = [odor_position[i]] 
    eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    compute_distance(wind_pos,odor_pos,i)

In [23]:
#for the section when only the length of the elements is equal to the window value
for i in range((len(eastwest))-1, -1, -1):
    
    odor_pos = [odor_position[i]]  
    if(i == 0):
        radius = np.zeros(1)
        wind_pos = np.array([[0,0]])
        distance = cdist(odor_pos,wind_pos).flatten()
        slope_sensor = np.arctan2(df.gps_linear_y[i],df.gps_linear_x[i])
        #manual appending for source position 
        #to avoid zeroth error
        relative_motion.append(wrapped_angular_diff(0,slope_sensor))
        distance_along_streakline.append(0)
        nearest_from_streakline.append(np.min(distance))
        
    else:
        eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
        northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
        wind_pos = np.vstack([eastwest,northsouth]).T   
        compute_distance(wind_pos,odor_pos,i)



In [24]:
## flip containers because above iteration is done in reverse order
df['mock'] = np.zeros(len(df))
distance_along_streakline = np.array(np.flip(distance_along_streakline)) 
nearest_from_streakline = np.array(np.flip(nearest_from_streakline)) 

# odor_presence = odor_presence[::-1]

## storing in the dataframe 
# df['odor_expected'] = odor_presence
df['distance_along_streakline'] =  distance_along_streakline
df['nearest_from_streakline'] = nearest_from_streakline
df['distance_from_source'] = distance_from_source
df['relative_motion']=relative_motion[::-1]

In [25]:
df.to_hdf(dir+'Data_expected.h5', key='df', mode='w')